# Wedge Task 3
It is useful to have summary files that allow you to quickly answer questions such as the following:
•	How have our sales-by-day changed over the last few months?
•	What is our most popular item in each department?
•	Which owners spend the most per month in each department?
The classic way to structure data to answer these questions is in a relational database. In this task, you will build the summary text files that hold this data and populate a relational database with the data.
Input
You will process your owner records in GBQ to build the summary tables.
Output
For this task, you will build a single SQLite database via Python (in a .db file) containing three tables:
1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 
2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 
3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.
You will submit your Python code that builds the database. 
You are welcome to generate these tables via queries in Google Big Query, export the text files, and store them locally on your machine. Then you will need to write a Python script that creates the database, creates the tables, and fills those tables. Obviously, it’d be great to do the whole thing in Python. 
Deliverable
The Python code that creates the summary tables. The Python code that builds the database. 

In [1]:
import sqlite3
import os
import pandas as pd
import re
import datetime 
import numpy as np
import pandas_gbq

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

# GBQ Set-up

In [2]:
# These first two values will be different on your machine. 
service_path = "C:\\Users\\jshay\\OneDrive\\Documents\\Applied Data Analytics\\Wedge\\"
service_file = 'umt-msba-gg-key.json'
gbq_proj_id = 'umt-msba'
dataset_id = 'transactions'

# And this should stay the same. 
private_key =service_path + service_file

In [3]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


# Table 1 sales_date_hour

In [6]:
table_1 = """SELECT  EXTRACT(DATE from datetime) AS date,
                     EXTRACT(HOUR from datetime) AS hour,
                     sum(total) AS spend,
                     count(distinct(date(datetime) || register_no ||
                                            emp_no || trans_no)) as Transactions,
             sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
             FROM `umt-msba.transactions.transArchive_*`
             WHERE department != 0 and
                   department != 15 and
                  (trans_status = '' or 
                   trans_status = ' ' or 
                   trans_status = 'V' or 
                   trans_status = 'R')
             GROUP BY date,hour
             ORDER BY date,hour"""

sales_date_hour = pandas_gbq.read_gbq(table_1,project_id = gbq_proj_id)

Downloading: 100%|██████████| 39312/39312 [00:02<00:00, 13798.20rows/s]


In [7]:
sales_date_hour

,date,hour,spend,Transactions,Items
0,2010-01-01,9,1006.28,36,245
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365
...,...,...,...,...,...
39307,2017-01-31,18,6992.52,255,1713
39308,2017-01-31,19,5395.00,208,1376
39309,2017-01-31,20,4635.16,160,1236
39310,2017-01-31,21,2910.39,115,833


# Table 2 sales_owner_ym

In [8]:
table_2 = """SELECT card_no,
                     EXTRACT(YEAR from datetime) AS year,
                     EXTRACT(MONTH from datetime) AS month,
                     sum(total) AS spend,
                     count(distinct(date(datetime) || register_no ||
                                            emp_no || trans_no)) as Transactions,
             sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
             FROM `umt-msba.transactions.transArchive_*`
             WHERE department != 0 and
                   department != 15 and
                  (trans_status = '' or 
                   trans_status = ' ' or 
                   trans_status = 'V' or 
                   trans_status = 'R')
             GROUP BY card_no, department, year, month
             ORDER BY Year, Month DESC"""

sales_owner_ym = pandas_gbq.read_gbq(table_2,project_id = gbq_proj_id)

Downloading: 100%|██████████| 5916674/5916674 [05:31<00:00, 17871.28rows/s]


In [9]:
sales_owner_ym

,card_no,year,month,spend,Transactions,Items
0,10310.0,2010,12,17.75,2,5
1,19339.0,2010,12,2.58,1,2
2,16130.0,2010,12,9.96,1,4
3,23261.0,2010,12,5.49,1,1
4,24162.0,2010,12,27.25,1,1
...,...,...,...,...,...,...
5916669,23071.0,2017,1,111.91,3,37
5916670,30725.0,2017,1,10.68,1,2
5916671,52808.0,2017,1,8.00,2,4
5916672,20985.0,2017,1,6.70,1,2


# Table 3 sales_prod_ym

In [10]:
table_3 = """SELECT upc,
                    tr.description AS prod_desc,
                    tr.department AS dept,
                    dd.dept_name AS dept_name,
                    EXTRACT(YEAR from datetime) AS year,
                    EXTRACT(MONTH from datetime) AS month,
                    sum(total) AS spend,
                    count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
               sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
               FROM `umt-msba.transactions.transArchive_*` tr
                LEFT JOIN `umt-msba.wedge_example.departments`dd
                    ON tr.department = dd.department
               WHERE tr.department != 0 and
                    tr.department != 15 and
                    trans_status != 'M' and
                    trans_status != 'C' and
                    trans_status != 'J' and
                   (trans_status = '' or 
                    trans_status = ' ' or 
                    trans_status = 'V' or 
                    trans_status = 'R')
                GROUP BY upc, prod_desc, year, month, tr.department, dd.dept_name
                ORDER BY spend DESC"""
sales_prod_ym = pandas_gbq.read_gbq(table_3,project_id = gbq_proj_id)

Downloading: 100%|██████████| 1183434/1183434 [02:02<00:00, 9632.72rows/s] 


In [11]:
sales_prod_ym

,upc,prod_desc,dept,dept_name,year,month,spend,Transactions,Items
0,0000000007025,Hot Bar Container,8.0,DELI,2017,1,46009.55,5725,6232
1,0000000007025,Hot Bar Container,8.0,DELI,2016,12,45076.97,5588,6117
2,0000000007025,Hot Bar Container,8.0,DELI,2016,11,42844.83,5393,5905
3,0000000000338,O.Blueberries WI pkg.,2.0,PRODUCE,2011,8,37502.50,6288,6644
4,0003338320038,O.Strawberries 16oz pkg.,2.0,PRODUCE,2010,5,37406.22,7645,8675
...,...,...,...,...,...,...,...,...,...
1183429,0000000008005,MEMBER DISCOUNT,1.0,PACKAGED GROCERY,2010,1,-12637.50,3382,3913
1183430,0000000008005,MEMBER DISCOUNT,1.0,PACKAGED GROCERY,2010,3,-13092.50,3592,4133
1183431,0000000008005,MEMBER DISCOUNT,1.0,PACKAGED GROCERY,2010,6,-43072.50,10311,12502
1183432,0000000008005,MEMBER DISCOUNT,1.0,PACKAGED GROCERY,2010,2,-44642.50,10548,13323


# Insert Tables into SQL Database

In [12]:
my_wedge = sqlite3.connect("task3_wedge.db")

In [13]:
sales_date_hour.to_sql(name = 'sales_date_hour', con = my_wedge)

39312

In [14]:
sales_owner_ym.to_sql(name = 'sales_owner_ym', con = my_wedge)

5916674

In [15]:
sales_prod_ym.to_sql(name = 'sales_prod_ym', con = my_wedge)

1183434

In [16]:
my_wedge.close()